In [5]:
import numpy as np

In [6]:
class Full_QR_Algorithm:
    def __init__(self, matrix):
        self.matrix = matrix.astype(np.float64)
        self.x_vector = [] # List down x vector each iteration. To be printed out later
        self.w_vector = [] # List down w vector each iteration. To be printed out later
        self.v_vector = [] # List down v vector each iteration. To be printed out later
        self.p_matrix = [] # List down projection matrix each iteration. To be printed out later
        self.H_hat_matrix = [] # List down Household Reflector hat each iteration. To be printed out later
        self.H_matrix = [] # List down Household Reflector matrices each iteration. To be printed out later
        self.similar_matrix_formula = []
        self.similar_matrix = []
        self.Givens_rotation_matrix = [] # List down all Rotation matrices that were used to rotate matrix A and Q
        self.R_matrix = [] # List down rotated matrix A
        self.Q_matrix = [] # List down rotated matrix Q
        self.A_matrix = [] # List down matrix A per iteration
        self.eigenvalues = [] # List down Eigenvalues per iteration
        self.num_rows, self.num_columns = self.GET_MATRIX_ROWS_COLUMNS(matrix)
        
    def GET_MATRIX_ROWS_COLUMNS(self, matrix):
        # Number of rows and columns in the matrix
        num_rows, num_cols = matrix.shape
        return num_rows, num_cols 
    
    def check_if_Upper_Hessenberg_Form(self, matrix):
        # Checks if the matrix is in Upper Hessenberg Form
        num_rows, num_columns = matrix.shape

        for i in range(num_rows):
            for j in range(num_columns):
                if i - j > 1 and matrix[i, j] != 0:
                    return False

        return True

    
    def Household_Reflector(self):
        matrix = self.matrix

        for i in range(self.num_columns - 1):  # Iterate over columns except the last one

            # Get x_vector
            x_vector = matrix[i + 1:, i]
            self.x_vector.append(x_vector)

            # Check if x_vector is all zeros
            if np.all(x_vector == 0):
                # Skip Householder transformation
                self.w_vector.append(np.zeros_like(x_vector))
                self.v_vector.append(np.zeros_like(x_vector))
                self.p_matrix.append(np.eye(len(x_vector)))
                self.H_hat_matrix.append(np.eye(len(x_vector)))
                self.H_matrix.append(np.eye(self.num_columns))
                continue

            # Normalize x_vector
            norm_x = np.sqrt(np.dot(x_vector, x_vector))
            w_vector = np.zeros_like(x_vector)
            if norm_x != 0:
                w_vector[0] = np.linalg.norm(x_vector)
            if x_vector[0] < 0:
                w_vector[0] = -w_vector[0]

            self.w_vector.append(w_vector)

            # Calculate v_vector
            v_vector = w_vector - x_vector
            self.v_vector.append(v_vector)

            # Calculate p_matrix
            p_matrix = np.outer(v_vector, v_vector) / np.dot(v_vector, v_vector)
            self.p_matrix.append(p_matrix)

            H_hat_matrix = np.eye(len(x_vector)) - 2 * p_matrix
            self.H_hat_matrix.append(H_hat_matrix)

            H_matrix = np.eye(self.num_columns)

            # Insert H_hat to H_matrix
            row_start, row_end = i + 1, i + 1 + H_hat_matrix.shape[0]
            col_start, col_end = i + 1, i + 1 + H_hat_matrix.shape[1]

            H_matrix[row_start:row_end, col_start:col_end] = H_hat_matrix
            self.H_matrix.append(H_matrix)

            # Compute similar matrix
            if i == 0:
                # Store the formula used "H1*A*H1"
                formula = "H1*A*H1"
                self.similar_matrix_formula.append(formula)

                # Do computation H1*A*H1
                computation = np.round(H_matrix @ matrix @ H_matrix, 10)
                self.similar_matrix.append(computation)
            else:
                # Store the formula used for H_i+1*(formula)*H_i+1
                formula = "H" + str(i + 1) + "(" + self.similar_matrix_formula[i - 1] + ")*H" + str(i + 1)
                self.similar_matrix_formula.append(formula)

                # Do computation H_i+1*(formula)*H_i+1
                computation = np.round(H_matrix @ self.similar_matrix[i - 1] @ H_matrix, 10)
                self.similar_matrix.append(computation)

            # Updating matrix
            matrix = computation

            # Check if matrix is already an upper Hessenberg
            validation = self.check_if_Upper_Hessenberg_Form(matrix)

            if validation == True:
                break

        # Return nothing
        return None

    
    def print_household_reflector_computations(self):
        
        print("________STEP 1A: HOUSEHOLDER REFLECTORS_________")
        print("")
        print("")
        
        for i in range(len(self.x_vector)):
            print("\033[1mIteration \033[0m" + str(i+1) + ":")
            print("\033[1mx vector: \033[0m" + str(self.x_vector[i]))
            print("\033[1mw vector: \033[0m" + str(self.w_vector[i]))
            print("\033[1mv vector: \033[0m" + str(self.v_vector[i]))
            print("")
            print("\033[1mP matrix: \033[0m")
            print("")
            print(self.p_matrix[i])
            print("")
            print("\033[1mH hat matrix: \033[0m")
            print("")
            print(self.H_hat_matrix[i])
            print("")
            print("\033[1mH matrix: \033[0m")
            print("")
            print(self.H_matrix[i])
            print("")
            print("\033[1mFormula: \033[0m" + str(self.similar_matrix_formula[i]))
            print("")
            print("\033[1mResulting matrix: \033[0m")
            print("")
            print(self.similar_matrix[i])
            print("")
            print("")
            
        return None
            
    def get_hessenberg_form_matrix(self):
        
        resulting_matrix = self.Household_Reflector()
        validation = self.check_if_Upper_Hessenberg_Form(self.similar_matrix[-1])
        
        if validation == True:
            solution = self.print_household_reflector_computations()
            similarity = self.check_similarity_matrix_Hessenberg()
            return resulting_matrix
        else:
            return "failure to find upper hessenberg form"
        
        return None


    def check_similarity_matrix_Hessenberg(self):
        print("")
        print("______STEP 1B: SIMILARITY TEST______")

        eigenvalues_A = np.linalg.eigvals(self.matrix)
        eigenvalues_H = np.linalg.eigvals(self.similar_matrix[-1])
        
        print("")
        print("\033[1mEigenvalues of matrix A: \033[0m")
        print("")
        print(eigenvalues_A)
        print("")
        print("\033[1mEigenvalues of it's Upper Hessenberg form: \033[0m")
        print("")
        print(eigenvalues_H)
        print("")

        # Check if the characteristic polynomials are the same

        if np.allclose(eigenvalues_A, eigenvalues_H, atol=1e-6):
            print("\033[1mSimilarity Result: \033[0m" + "Matrix A and it's upper Hessenberg form H are similar.")
            print("")
            print("\033[1mProceeding to STEP 2\033[0m")
            print("")
            QR_Solution = self.QR_algorithm(self.similar_matrix[-1])
        else:
            print("\033[1mSimilarity Result: \033[0m" + "Matrix A and it's upper Hessenberg form H are not similar.")
        
        return None
    
    def print_QR_computations(self):
        print("")
        print("")
        print("")
        print("___________STEP 2: QR ALGORITHM____________")
        print("")
        print("")
        
        print("\033[1mHess matrix: \033[0m")
        print("")
        print(self.similar_matrix[-1])
        print("")
        print("\033[1mInitial eigenvalues: \033[0m")
        print("")
        print(self.eigenvalues[0])
        
        for i in range(len(self.R_matrix)):
            print("")
            print("\033[1mIteration \033[0m" + str(i+1) + ":")
            print("")
            print("\033[1mGivens Rotation:\033[0m")
            print("")
            print("\033[1mR matrix:\033[0m")
            print("")
            print(self.R_matrix[i])
            print("")
            print("\033[1mQ matrix:\033[0m")
            print("")
            print(self.Q_matrix[i])
            print("")
            print("\033[1mA prime:\033[0m")
            print("")
            print(self.A_matrix[i])
            print("")
            print("\033[1mEigenvalues:\033[0m")
            print("")
            print(self.eigenvalues[i+1])
            if i+1 == len(self.R_matrix):
                print("")
                print("\033[1mThe difference of eigenvalues are lower than the tolerance limit. Breaking...\033[0m")
                print("")
        
        return None
            
    def Givens_Rotation_QR(self, matrix):
        n = matrix.shape[0]
        A_matrix = matrix.astype(np.complex128)
        Q_matrix = np.identity(n)
        Q_matrix = Q_matrix.astype(np.complex128)
        
        for j in range(n):
            for i in range(n - 1, j, -1):
                if A_matrix[i, j] != 0:
                    print("Found nonzero")
                    print("a = " + str(A_matrix[i-1,j]))
                    print("b = " + str(A_matrix[i, j]))
                    # Ensure that the elements are complex numbers and compatible for arctan2
                    if isinstance(A_matrix[i, j], complex) and isinstance(A_matrix[i-1, j], complex):
                        a = A_matrix[i-1, j]
                        b = A_matrix[i, j]
                        r = np.sqrt(abs(a)**2 + abs(b)**2)
                        cos = abs(a) / r
                        sin = abs(b) / r
                        print("cos = " + str(cos))
                        print("sin = " + str(sin))
                        print("r = " + str(r))
                        G_matrix = np.identity(n, dtype=complex)
                        
                        if b == 0:
                            c = 1.0 if a == 0 else np.sign(a)
                            s = 0
                            r = abs(a)
                        elif a == 0:
                            c = 0
                            s = -np.sign(b)
                            r = abs(b)
                        elif abs(a) > abs(b):
                            t = b / a
                            u = np.sign(a) * np.sqrt(1 + t * t)
                            c = 1 / u
                            s = -c * t
                            r = a * u
                        else:
                            t = a / b
                            u = np.sign(b) * np.sqrt(1 + t * t)
                            s = -1 / u
                            c = t / u
                            r = b * u

                        G_matrix[i, i] = G_matrix[i-1, i-1] = c
                        G_matrix[i, i-1] = s
                        G_matrix[i-1, i] = -s
                        
                        self.Givens_rotation_matrix.append(G_matrix)
                        
                        print("G matrix:")
                        print(G_matrix)
                        print("A_matrix")
                        print(A_matrix)
                        print("Q_matrix")
                        print(Q_matrix)
                        print("")

                        # Apply the rotation to A and Q
                        threshold = 1e-10
                        A_matrix = G_matrix @ A_matrix
                        A_matrix[np.abs(A_matrix) < threshold] = 0
                        Q_matrix = Q_matrix @ G_matrix.T
                        
                        print("A_matrix")
                        print(A_matrix)
                        print("Q_matrix")
                        print(Q_matrix)
                        
                        tolerance = 1e-10
                        is_orthogonal = np.allclose(np.dot(Q_matrix.T, Q_matrix), np.eye(Q_matrix.shape[1]), atol=tolerance)
                        if is_orthogonal:
                            print("Q is orthogonal.")
                        else:
                            print("Q is not orthogonal.")

                    else:
                        print("")
                        print("\033[1mElements are not complex numbers\033[0m")

                                      
        tolerance = 1e-10
        is_orthogonal = np.allclose(np.dot(Q_matrix.T, Q_matrix), np.eye(Q_matrix.shape[1]), atol=tolerance)
        if is_orthogonal:
            print("Q is orthogonal.")
        else:
            print("Q is not orthogonal.")
        print("Resulting_Q_matrix")
        print(Q_matrix)
        self.R_matrix.append(A_matrix)
        self.Q_matrix.append(Q_matrix)
        return Q_matrix, A_matrix
    
    def schur_decomposition(self, matrix):
        A_matrix = matrix
        n = A_matrix.shape[0]
        Q_matrix = np.identity(n)

        for k in range(n - 1):
            H_matrix = A_matrix.copy()
            for i in range(k+1, n):
                for j in range(k, n):
                    Aij, Ajj, Aii = H_matrix[i, j], H_matrix[j, j], H_matrix[i, i]
                    if Aij != 0:
                        phi = 0.5 * np.arctan2(2 * Aij, Ajj - Aii)
                        cos, sin = np.cos(phi), np.sin(phi)
                        G_matrix = np.identity(n)
                        G_matrix[i, i] = G_matrix[j, j] = cos
                        G_matrix[i, j] = -sin
                        G_matrix[j, i] = sin
                        A_matrix = np.dot(G_matrix.T, np.dot(A_matrix, G_matrix))
                        Q_matrix = np.dot(Q_matrix, G_matrix)

        return Q_matrix, A_matrix
    
    def QR_algorithm(self, matrix, max_iterations=1000, tol=1e-6):
        n = matrix.shape[0] 
        A_matrix = matrix

        # Initialize Variables
        eigenvalues = np.zeros(n, dtype=complex)
        self.eigenvalues.append(eigenvalues)
        
        eigenvectors = np.identity(n, dtype=complex)
        shift = 0.0 + 0.0j  # Initial guess for the eigenvalue (complex)
        
 
        for iteration in range(max_iterations):

            # Compute for the shift 
            A_matrix_2x2 = A_matrix[-2:, -2:] # Bottom most right of matrix A [(a,b),(b,c)]. We expect it to be symmetric but non-symmetric should be fine
            a = A_matrix_2x2[0,0]
            b = A_matrix_2x2[0,1]
            c = A_matrix_2x2[1,0]

            print("")
            print("\033[1mBottom most right 2x2 of matrix A:\033[0m")
            print("")
            print(A_matrix_2x2)

            l = (a-c)/2

            if l < 0:
                shift = c + (b*b)/(abs(l)+(l*l + b*b)**0.5)
            elif l > 0:
                shift = c - (b*b)/(abs(l)+(l*l + b*b)**0.5)
            else:
                shift = c + (b*b)/(abs(l)+(l*l + b*b)**0.5)

            shift = complex(shift, 0.0)

            print("")
            print("Computed shift: " + str(shift))
            print("")
            
            # Compute QR decomposition of Hessenberg matrix H
            Q_matrix, R_matrix = self.Givens_Rotation_QR(A_matrix - shift * np.identity(n))

            # Compute shifted matrix A'
            A_prime = np.dot(R_matrix, Q_matrix) + shift * np.identity(n)
            self.A_matrix.append(A_prime)

            # Update Hessenberg matrix for the next iteration
            A_matrix = A_prime

            # Check for convergence
            eigenvalues_new = np.diag(A_matrix)
            self.eigenvalues.append(eigenvalues_new)
            
            if np.all(np.abs(eigenvalues - eigenvalues_new) < tol):
                # Print QR computations
                print_QR = self.print_QR_computations()
                break

            eigenvalues = eigenvalues_new
        
        if iteration == max_iterations - 1:
            print("")
            print("\033[1mQR algorithm did not converge to the desired tolerance.\033[0m")
            print("\033[1mIterations: \033[0m" + str(iteration + 1))
            
        # Check for complex eigenvalues
        print("")
        print("___________STEP 3: CHECKING FOR IMAGINARY EIGENVALUES____________")
        print("")
        if any(np.iscomplex(eigenvalues)):
            print("")
            print("\033[1mComplex eigenvalues detected. Applying Schur decomposition.\033[0m")
            print("")
            Q_schur, A_schur = self.schur_decomposition(A_matrix)
            eigenvalues_schur = np.diag(A_schur)
            eigenvectors_schur = Q_schur
            print("\033[1mEigenvectors:\033[0m")
            print("")
            print(eigenvectors_schur)
            print("")
            print("\033[1mEigenvalues:\033[0m")
            print("")
            print(eigenvalues_schur)
            return eigenvalues_schur, eigenvectors_schur
        else: 
            print("")
            print("\033[1mNo complex eigenvalues found\033[0m")
            print("")            

        # Find Eigenvalues of A
        eigenvalues = eigenvalues

        # if A is symmetric, Find Eigenvectors of A
        print("")
        print("___________STEP 4: CHECKING FOR SYMMETRIC MATRIX____________")
        print("")
        if np.allclose(A_matrix, A_matrix.T, rtol=tol):
            print("")
            print("\033[1mMatrix A is symmetric. Finding Eigenvectors of A.\033[0m")
            print("")
            eigenvectors = np.linalg.inv(eigenvectors)
            print("\033[1mEigenvectors:\033[0m")
            print("")
            print("___________STEP 5: FINAL RESULT____________")
            print("")
            print(eigenvectors)
            print("")
            print("\033[1mEigenvalues:\033[0m")
            print("")
            print(eigenvalues)
        else:
            print("")
            print("\033[1mMatrix A is not symmetric\033[0m")
            print("")
            print("")
            print("___________STEP 5: FINAL RESULT____________")
            print("")
            print("\033[1mEigenvalues:\033[0m")
            print("")
            print(eigenvalues)

        return eigenvalues, eigenvectors




In [7]:
A = np.random.rand(4,4)

print("")
print("\033[1minput matrix A:\033[0m")
print("")
print(A)



QR_algo = Full_QR_Algorithm(A)
QR_algo.get_hessenberg_form_matrix()


input matrix A:

[[0.46004246 0.49623488 0.9045754  0.76069234]
 [0.95888152 0.61876512 0.41860353 0.42009259]
 [0.29643243 0.42501873 0.71240879 0.87044467]
 [0.22941229 0.03885658 0.79791553 0.22655128]]
________STEP 1A: HOUSEHOLDER REFLECTORS_________


Iteration 1:
x vector: [0.95888152 0.29643243 0.22941229]
w vector: [1.02954162 0.         0.        ]
v vector: [ 0.0706601  -0.29643243 -0.22941229]

P matrix: 

[[ 0.03431629 -0.1439633  -0.11141477]
 [-0.1439633   0.60395316  0.46740593]
 [-0.11141477  0.46740593  0.36173054]]

H hat matrix: 

[[ 0.93136742  0.28792661  0.22282954]
 [ 0.28792661 -0.20790633 -0.93481187]
 [ 0.22282954 -0.93481187  0.27653891]]

H matrix: 

[[ 1.          0.          0.          0.        ]
 [ 0.          0.93136742  0.28792661  0.22282954]
 [ 0.          0.28792661 -0.20790633 -0.93481187]
 [ 0.          0.22282954 -0.93481187  0.27653891]]

Formula: H1*A*H1

Resulting matrix: 

[[ 0.46004246  0.89213305 -0.75629195 -0.524671  ]
 [ 1.02954162  1.

[[ 1.54122528e+00+0.j -2.75772826e-01+0.j -2.45294128e-01+0.j
  -6.79867848e-01+0.j]
 [ 7.59877084e-04+0.j -9.57374622e-01+0.j  1.76095952e-01+0.j
   8.97566772e-02+0.j]
 [ 0.00000000e+00+0.j -1.49113543e-03+0.j -6.71678684e-01+0.j
   6.15083357e-01+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
  -1.47431219e-01+0.j]]
Q_matrix
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]]

A_matrix
[[ 1.54122547e+00+0.j -2.76244811e-01+0.j -2.45207277e-01+0.j
  -6.79823512e-01+0.j]
 [ 0.00000000e+00+0.j -9.57238541e-01+0.j  1.76216869e-01+0.j
   9.00918645e-02+0.j]
 [ 0.00000000e+00+0.j -1.49113543e-03+0.j -6.71678684e-01+0.j
   6.15083357e-01+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
  -1.47431219e-01+0.j]]
Q_matrix
[[ 9.99999878e-01+0.j -4.93034340e-04+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j]
 [ 4.93034340e-04+0.j  9.99999878e-01+0.j  0.00000000e+00+0.j
   0.00000000e+00+

[[ 1.00000000e+00+0.j  4.14363795e-06+0.j -1.89711879e-10+0.j
   0.00000000e+00+0.j]
 [ 4.14363795e-06+0.j -9.99999999e-01+0.j  4.57838936e-05+0.j
   0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j -4.57838936e-05+0.j -9.99999999e-01+0.j
   0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   1.00000000e+00+0.j]]

A prime:

[[ 2.10439775e+00+0.j  2.77790953e-01+0.j  2.43818120e-01+0.j
  -6.79840498e-01+0.j]
 [ 3.95934160e-06+0.j -3.93130065e-01+0.j  1.77631367e-01+0.j
  -9.31184112e-02+0.j]
 [ 0.00000000e+00+0.j -3.07535174e-05+0.j -1.09325536e-01+0.j
  -6.14613655e-01+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   4.15825505e-01+0.j]]

Eigenvalues:

[ 2.10439775+0.j -0.39313007+0.j -0.10932554+0.j  0.41582551+0.j]

Iteration 27:

Givens Rotation:

R matrix:

[[ 1.54202084+0.j  0.2777885 +0.j  0.24381858+0.j -0.67984074+0.j]
 [ 0.        +0.j  0.95550769+0.j -0.17760912+0.j  0.09313645+0.j]
 [ 0.        +0.j  0.        +0.j  0.67170816+0